<a href="https://colab.research.google.com/github/iamronsuez/aprendizaje-automatico/blob/main/Actividad_3_grupal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Aprendizaje Automático_

_Máster Universitario en Inteligencia Artificial_

# Caso grupal: Detección de anomalías y técnicas de agrupamiento

## Objetivos

- Entender el método de clustering K-means.
- Generar agrupamientos con los métodos de clustering indicados y realizar una comparativa de ambos.
- Investigar sobre una técnica de detección de anomalías con un caso de uso concreto.


## Descripción de la actividad

El conjunto de datos con el cual vamos a trabajar se encuentra en el siguiente enlace http://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+consumption

Se trata de un dataset que contiene mediciones del consumo de energía eléctrica en un hogar con una tasa de muestreo de un minuto durante un período de casi 4 años. Están disponibles diferentes magnitudes eléctricas y algunos valores de submedición. Debe descargarse el fichero comprimido que se encuentra en la carpeta 'Data Folder'.

### Tareas que se deben realizar
- Clustering
  - Debes completar el código pedido en este notebook y contestar las preguntas finales.
- Detección de anomalías
  - Buscar dos artículos científicos (https://scholar.google.es/) con un caso de uso de detección de anomalías. Los artículos deben estar en revistas científicas, y deben ser posteriores a 2015. No debe utilizar técnicas de Deep Learning.
  - Para cada artículo indicar:
    - Objetivo: cuál es el objetivo de la investigación, es decir a qué problema real está aplicando la detección de anomalías.
    - Técnicas de detección de anomalías empleadas.
    - Principales resultados de la aplicación y de la investigación.


### Clustering

Comenzamos importando las librerías necesarias y cargando el dataset.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

# se carga el dataset, con separados ;, se transforman las dos columnas Date y Time a una única columna con tipo datetime
# en el fichero CSV existen NaN identificados por ?
df = pd.read_csv('./household_power_consumption.txt', sep=';', parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True,low_memory=False,na_values=['nan','?'])


<ipython-input-4-072f94f28fc1>:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## cuántas instancias tiene el dataset
## ref (2075259)

In [ ]:
### PON AQUÍ TU CÓDIGO
len(df)

In [ ]:
## cuál es el tipo de datos de la variable 'Voltage'.
## ref D

In [ ]:
### PON AQUÍ TU CÓDIGO
df['Voltage'].dtype

La variable Voltage es de tipo Float64 o Dobule


In [ ]:
## ¿es una variable continua o categórica?
## ref Continuous
df['Voltage'].nunique()


Es una variable continua, ya que las variables continuas tienden a tener mas de dos valores

In [ ]:
# ¿cuáles son los valores mínimo y máximo de la variable 'Voltage'?

In [ ]:
### PON AQUÍ TU CÓDIGO

min_value = df['Voltage'].min()
min_value

In [ ]:
max_value = df['Voltage'].max()
max_value

Valor minimo: 223.2
Valor maximo: 254.15

In [ ]:
# ¿existe alguna variable con más del 30% de valores faltantes?

In [ ]:
## PON AQUÍ TU CÓDIGO
missing_pcts = df.isna().mean()

missing_pcts_above_30 = missing_pcts[missing_pcts > 0.3]

missing_pcts_above_30

Existen variables con datos faltantes, pero ninguna sobre el 30%

In [ ]:
# ¿cuántas variables categóricas hay en el dataset?
## Ref Time

In [ ]:
### PON AQUÍ TU CÓDIGO
for column in df.columns:
    print(f"{column}: {df[column].nunique()} valores únicos")

Indica las categorías de cada una de ellas si las hubiera.

Segun el dataset, la variable categorica definida es Time, que fue convertida en dt.

Realizando el analisis de los valores unicos del dataframe, podemos ver que no existe un criterio claro para decidir que una variable es categorica ya que todas tienen un comportamiento como variables continuas.

En conclusion podemos decir que ninguna es categorica.


Para los algoritmos de clustering el dataset no puede tener instancias con datos faltantes. ¿Tiene este dataset datos faltantes?

In [ ]:
### PON AQUÍ TU CÓDIGO
df.isna().any().any()

Si, efectivamente tiene datos faltantes, ya lo vimos en el % de datos faltantes

Existen diferentes alternativas para completar los datos faltantes, algunas de ellas las hemos estudiado en la asignatura.
Sin embargo, en esta actividad vamos a prescindir de estos datos.

In [ ]:
df.dropna(inplace=True)
df.isna().any().any()

Se verifica que no tiene datos faltanes

In [ ]:
# Calcula las correlaciones de Pearson de las variables

In [ ]:
### PON AQUÍ TU CÓDIGO
df.corr(method='pearson',numeric_only=True)



¿Cuál es la variable que tiene mayor correlación con 'Global_active_power'? ¿Son variables dependientes o independientes?

In [ ]:
gap_corr = df.corr(method='pearson',numeric_only=True)['Global_active_power'].sort_values(ascending=False)
gap_corr



La variable con mayor correlacion es Global_intensity

¿Podemos eliminar alguna variable?

In [ ]:
# eliminamos variable dt
dataFrame = df.drop(columns='dt')

¿Es necesario hacer algún tratamiendo adicional sobre los datos?

Sí, es común que los datos requieran tratamientos adicionales antes de aplicar métodos de clustering como el k-means. Uno de estos tratamientos es la estandarización de los datos, que se puede realizar mediante el uso de técnicas como el Standard Scaler.

El Standard Scaler ajusta los datos de manera que tengan una media de 0 y una desviación estándar de 1. Este paso es importante porque el k-means utiliza distancias euclidianas para determinar la similitud entre los puntos de datos, y las variables con escalas más grandes pueden influir desproporcionadamente en el resultado del clustering.

In [ ]:
### PON AQUÍ TU CÓDIGO EN CASO DE QUE SEA NECESARIO HACER ALGÚN TRATAMIENTO ADICIONAL
df.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['Global_active_power_T', 'Global_reactive_power_T', 'Voltage_T',
    'Global_intensity_T', 'Sub_metering_1_T',
    'Sub_metering_2_T', 'Sub_metering_3_T']] = scaler.fit_transform(df[['Global_active_power',  'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']])
df.describe()

Los métodos de clustering necesitan saber de antemano cuántos clusters deben crear.
¿Cuál es uno de los métodos más populares para saber cuál es el número apropiado de clústers?

Uno de los métodos más populares para determinar el número apropiado de clusters en un análisis de clustering es el método del codo, o "Elbow Method" en inglés. Este método implica graficar la variación explicada como una función del número de clusters y buscar el punto en el cual el incremento en la variación explicada se reduce significativamente, creando un ángulo en el gráfico que se asemeja a un codo. Este punto es considerado como una indicación del número adecuado de clusters, ya que añadir más clusters más allá de este punto no mejora sustancialmente la varianza explicada por los modelos.

Crea una lista de 20 modelos de KMeans y entrénalos.

Haciendo uso del método indicado en la respuesta anterior, indica cuál es el número de clusters que parece más prometedor.

El modelo de KMeans debe tener los siguientes parámetros:
- n_clusters (entre 1 y 20)
- init: k-means++
- algorithm: algoritmo de Lloyd

In [ ]:
def get_kmeans(data, max_k):
    means = []
    inertias = []
    k_means= []
    for k in range(1, max_k):
        kmeans = KMeans(n_clusters=k,init='k-means++',n_init=10)
        kmeans.fit(data)

        means.append(k)
        inertias.append(kmeans.inertia_)
        k_means.append(kmeans)

    return [k_means, means, inertias]

def plot_inertias(means, inertias):
    fig = plt.subplots(figsize=(10, 5))
    plt.plot(means, inertias, 'o-')
    plt.xlabel('N clusters')
    plt.ylabel('Inertia')
    plt.grid(True)
    plt.show()

In [ ]:
from sklearn.cluster import KMeans
X=df[['Global_active_power_T', 'Global_reactive_power_T', 'Voltage_T',
    'Global_intensity_T', 'Sub_metering_1_T',
    'Sub_metering_2_T', 'Sub_metering_3_T']].copy()

[k_means, means, inertias] = get_kmeans(X,20)
plot_inertias(means, inertias)

In [ ]:
print(k_means, means, inertias)

A partir de estos resultados, ¿cómo sabemos cuál es el número óptimo de clústers? ¿qué número es?

_indica aquí tu respuesta_

In [ ]:
# Después de decidir el número de clusters, entrena el modelo con dicho número
n = 4


In [ ]:
kmeans = KMeans(n_clusters = n, n_init = 10, init='k-means++').fit(X)
centroids = kmeans.cluster_centers_
print(centroids)

In [ ]:
kmeansmodel = KMeans(n_clusters = n, init='k-means++', random_state=0, n_init = 10)
y_kmeans = kmeansmodel.fit_predict(X)

Ahora toca interpretar lo que Kmeans ha realizado. Recordemos que el objetivo de KMeans es encontrar patrones en los datos.

Lo primero que vamos a realizar es un gráfico para visualizar los clústers que ha creado en base a las variables utilizadas para crearlos.

In [ ]:
import plotly.express as px

clusters = pd.DataFrame(X,columns=X.columns)
clusters['label'] = kmeansmodel.labels_
polar = clusters.groupby("label").mean().reset_index()
polar = pd.melt(polar,id_vars=["label"])
fig4 = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=800,width=600)
fig4.show()

In [ ]:
import plotly.express as px

clusters = X[['Sub_metering_2_T','Global_intensity_T']].copy()
clusters = pd.DataFrame(clusters,columns=clusters.columns)
clusters['label'] = y_kmeans
clusters

En base a la figura de la representación de los clusters, ¿cuál es la variable que más nos ayuda a diferenciar los usos?

_introduce aquí tu respuesta_

In [ ]:
pie = clusters.groupby('label').size().reset_index()
pie.columns = ['label','value']
px.pie(pie,values ='value',names ='label')

¿Qué representa el gráfico de la celda anterior?

_introduce aquí tu respuesta_

¿Qué características tiene el consumo del cluster 0?

_introduce aquí tu respuesta_

Repite lo mismo pero esta vez utiliza para el entrenamiento de KMeans el algortimo de 'Elkan', ¿existen diferencias en el número de clústers? ¿y en la importancia de las variables?

In [ ]:
def get_kmeans_elkan(data, max_k):
    means = []
    inertias = []
    k_means= []
    for k in range(1, max_k):
        kmeans = KMeans(n_clusters=k,init='k-means++',n_init=10, algorithm='elkan')
        kmeans.fit(data)

        means.append(k)
        inertias.append(kmeans.inertia_)
        k_means.append(kmeans)

    return [k_means, means, inertias]

In [ ]:
X = df[['Global_active_power_T', 'Global_reactive_power_T', 'Voltage_T',
    'Global_intensity_T', 'Sub_metering_1_T',
    'Sub_metering_2_T', 'Sub_metering_3_T']].copy()

X

In [ ]:
[k_means_elkan, means_elkan, inertias_elkan] = get_kmeans_elkan(X,20)
plot_inertias(means_elkan, inertias_elkan)

In [ ]:
n = 4
kmeans_elkan = KMeans(n_clusters = n, n_init = 10, init='k-means++', algorithm='elkan').fit(X)
centroids = kmeans_elkan.cluster_centers_
print(centroids)

kmeansmodel_elkan = KMeans(n_clusters = n, init='k-means++', random_state=0, algorithm='elkan', n_init = 10)
y_kmeans_elkan = kmeansmodel_elkan.fit_predict(X)



In [ ]:

import plotly.express as px
clusters = pd.DataFrame(X,columns=X.columns)
clusters['label'] = kmeansmodel_elkan.labels_
polar = clusters.groupby("label").mean().reset_index()
polar = pd.melt(polar,id_vars=["label"])
fig4 = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=800,width=600)
fig4.show()

In [ ]:
pie = clusters.groupby('label').size().reset_index()
pie.columns = ['label','value']
px.pie(pie,values ='value',names ='label')

### Detección de anomalías

Artículo 1

_introduce aquí la referencia APA del artículo_

título, autores, revista, año de publicación

Objetivo: cuál es el objetivo de la investigación, es decir a qué problema real está aplicando la detección de anomalías.
Técnicas de detección de anomalías empleadas.
Principales resultados de la aplicación y de la investigación.

Artículo 2

_introduce aquí la referencia APA del artículo_

Kumari, R., Sheetanshu, Singh, M. K., Jha, R., & Singh, N. K. (2016).
    Anomaly detection in network traffic using K-mean clustering. *2016 3rd International Conference on Recent Advances in Information Technology (RAIT).*


título, autores, revista, año de publicación

Objetivo: cuál es el objetivo de la investigación, es decir a qué problema real está aplicando la detección de anomalías.
Técnicas de detección de anomalías empleadas.
Principales resultados de la aplicación y de la investigación.